In [1]:
import ipycanvas as ipc
import numpy as np
import random
from math import pi

In [48]:
EAST = 1
SOUTH = 0
NORTH = 2
WEST = 3

X = 0
Y = 1

OFFSETS = [[0, 1], [1, 0], [0, -1], [-1, 0]]
passage_width = 512 / 32
wall_width = passage_width / 2
canvas = ipc.Canvas(width = 512, height = 512)

def add_passage(passages, a, b):
    if ((b[0] - a[0] == 1) and (b[1] == a[1])):
        passages[a[0]][a[1]][EAST] = True
    elif ((b[0] - a[0] == -1) and (b[1] == a[1])):
        passages[a[0]][a[1]][WEST] = True
    elif ((b[1] - a[1] == 1) and (b[0] == a[0])):
        passages[a[0]][a[1]][SOUTH] = True
    elif ((b[1] - a[1] == -1) and (b[0] == a[0])):
        passages[a[0]][a[1]][NORTH] = True
        
def add_to_front(start, list):
    new_list = np.zeros((len(list) + 1, 2), dtype=int)
    new_list[0][0] = start[0]
    new_list[0][1] = start[1]
    for i in range(len(list)):
        new_list[i+1][0] = list[i][0]
        new_list[i+1][1] = list[i][1]
    return new_list

def choose_randomly_from(list, n):
    if n == 0:
        return None
    index = random.randint(0, n - 1)
    return list[index]

def contains(pair, list, n):
    for i in range(n):
        if ((pair[0] == list[i][0]) and (pair[1] == list[i][1])):
            return i
    return -1
 
def draw_maze(passages, passage_width):
    width = len(passages)
    canvas.line_width = 0.7
    canvas.stroke_style = 'white'
    canvas.line_cap = 'square'
    for x in range(width):
        for y in range(width):
            draw_rect(x * (passage_width + wall_width) + wall_width, y * (passage_width + wall_width), passage_width, wall_width, 'black')
            draw_rect(x * (passage_width + wall_width), y * (passage_width + wall_width) + wall_width, wall_width, passage_width, 'black')
    for i in range(width):
        draw_rect(i * (passage_width + wall_width) + wall_width, 512 - wall_width, passage_width, wall_width, 'black')
        draw_rect(512 - wall_width, i * (passage_width + wall_width) + wall_width, wall_width, passage_width, 'black')
    for x in range(width):
        for y in range(width):
            if passages[x][y][SOUTH] or (y + 1 < width and passages[x][y + 1][NORTH]):
                draw_rect(x * (passage_width + wall_width) + wall_width, (y + 1) * (passage_width + wall_width), passage_width, wall_width, 'pink')
            if passages[x][y][EAST] or (x + 1 < width and passages[x + 1][y][WEST]):
                draw_rect((x + 1) * (passage_width + wall_width), y * (passage_width + wall_width) + wall_width, wall_width, passage_width, 'pink')
    # canvas.begin_path()
    # canvas.move_to(0.5, 0)
    # canvas.line_to(0.5, 0.5)
    # canvas.move_to(width - 0.5, width - 0.5)
    # canvas.line_to(width, width - 0.5)
    # canvas.stroke()
    # canvas.close_path()
    draw_rect(wall_width, 0, passage_width, wall_width, 'pink')
    draw_rect(512 - wall_width, 512 - wall_width - passage_width, wall_width, passage_width, 'pink')

def expand_location(passages, unexplored, n, here, direction):
    there = np.zeros(2, dtype=int)
    there[X] = here[X] + OFFSETS[direction][X]
    there[Y] = here[Y] + OFFSETS[direction][Y]  
    if contains(there, unexplored, n) != -1:
        add_passage(passages, here, there)
        return there
    else:
        return None

def remove(pair, list, n):
    for i in range(n):
        if ((pair[0] == list[i][0]) and (pair[1] == list[i][1])):
            list[i][0] = list[n-1][0]
            list[i][1] = list[n-1][1]

def expand_maze(passages, done, frontier, unexplored, counts, last_explored_location):
    here = np.zeros(2, dtype=int)
    if last_explored_location is None:
        here = choose_randomly_from(frontier, counts[1])
    else:
        here = last_explored_location
    direction = random.randint(0, 3)
    for i in range(4):
        there = expand_location(passages, unexplored, counts[2], here, direction)
        if there is not None:
            frontier[counts[1]] = there
            counts[1] += 1
            remove(there, unexplored, counts[2])
            counts[2] -= 1
            return there
        direction = (direction + 1) % 4
    done[counts[0]] = here
    counts[0] += 1
    remove(here, frontier, counts[1])
    counts[1] -= 1
    return None

def draw_solution(path, width):
    canvas.line_width = 2
    canvas.stroke_style = 'red'
    canvas.begin_path()
    canvas.move_to(0.67 * (passage_width + wall_width), 0)
    canvas.line_to(0.67 * (passage_width + wall_width), 0.67 * (passage_width + wall_width))
    canvas.move_to((width - 0.33) * (passage_width + wall_width), (width - 0.33) * (passage_width + wall_width))
    canvas.line_to(512, (width - 0.33) * (passage_width + wall_width))
    canvas.stroke()
    canvas.close_path()
    for i in range(len(path) - 1):
        canvas.begin_path()
        canvas.move_to((path[i][0] + 0.67) * (passage_width + wall_width), (path[i][1] + 0.67) * (passage_width + wall_width))
        canvas.line_to((path[i + 1][0] + 0.67) * (passage_width + wall_width), (path[i + 1][1] + 0.67) * (passage_width + wall_width))
        canvas.stroke()
        canvas.close_path()

def solve(passages, start, goal):
        if ((start[0] == goal[0]) and (start[1] == goal[1])):
            return [goal]
        for d in range(4):
            if (passages[start[0]][start[1]][d]):
                next = [start[X] + OFFSETS[d][X], start[Y] + OFFSETS[d][Y]]
                restOfPath = solve(passages, next, goal)
                if (restOfPath is not None):
                    return add_to_front(start, restOfPath)
        return None
    
def draw_square(x, y, width, color):
    canvas.fill_style = color
    canvas.fill_rect(x, y, width)
    
def draw_rect(x, y, width, height, color):
    canvas.fill_style = color
    canvas.fill_rect(x, y, width, height)

def main():
    width = 21
    canvas.clear()
    canvas.fill_style = 'pink'
    canvas.fill_rect(0, 0, 512, 512)
    # canvas.scale(passage_width, passage_width)
    passages = np.full((width, width, 4), False)
    done = np.full(width * width, None)
    frontier = np.full(width * width, None)
    frontier[0] = np.zeros(2, dtype=int)
    unexplored = np.full(width * width, None)
    counts = [0, 1, width * width - 1]
    i = 0
    for x in range(width):
        for y in range(width):
            # draw_square(x * (passage_width + wall_width) + wall_width, y * (passage_width + wall_width) + wall_width, passage_width, 'white')
            if (x != 0 or y != 0):
                unexplored[i] = [x, y]
                i += 1
    for x in range(width + 1):
        for y in range(width + 1):
            draw_square(x * (passage_width + wall_width), y * (passage_width + wall_width), wall_width, 'black')
    last_explored_location = None
    while (counts[2] > 0):
        last_explored_location = expand_maze(passages, done, frontier, unexplored, counts, last_explored_location)
    draw_maze(passages, passage_width)
    
    solution = solve(passages, [0, 0], [width - 1, width - 1])
    draw_solution(solution, width)
    
main()
canvas



Canvas(height=512, width=512)

In [3]:

EAST = 1
SOUTH = 0
NORTH = 2
WEST = 3

X = 0
Y = 1

OFFSETS = [[0, 1], [1, 0], [0, -1], [-1, 0]]

canvas = ipc.Canvas(width = 512, height = 512)

def add_passage(passages, a, b):
    if ((b[0] - a[0] == 1) and (b[1] == a[1])):
        passages[a[0]][a[1]][EAST] = True
    elif ((b[0] - a[0] == -1) and (b[1] == a[1])):
        passages[a[0]][a[1]][WEST] = True
    elif ((b[1] - a[1] == 1) and (b[0] == a[0])):
        passages[a[0]][a[1]][SOUTH] = True
    elif ((b[1] - a[1] == -1) and (b[0] == a[0])):
        passages[a[0]][a[1]][NORTH] = True
        
def add_to_front(start, list):
    new_list = np.zeros((len(list) + 1, 2), dtype=int)
    new_list[0][0] = start[0]
    new_list[0][1] = start[1]
    for i in range(len(list)):
        new_list[i+1][0] = list[i][0]
        new_list[i+1][1] = list[i][1]
    return new_list

def choose_randomly_from(list, n):
    if n == 0:
        return None
    index = random.randint(0, n - 1)
    return list[index]

def contains(pair, list, n):
    for i in range(n):
        if ((pair[0] == list[i][0]) and (pair[1] == list[i][1])):
            return i
    return -1
 
def draw_maze(passages):
    width = len(passages)
    canvas.line_width = 0.7
    canvas.stroke_style = 'white'
    canvas.line_cap = 'square'
    for x in range(width):
        for y in range(width):
            if passages[x][y][SOUTH] or (y + 1 < width and passages[x][y + 1][NORTH]):
                canvas.begin_path()
                canvas.move_to(x + 0.5, y + 0.5)
                canvas.line_to(x + 0.5, y + 1.5)
                canvas.stroke()
                canvas.close_path()
                # draw_square(x + 0.5, y + 1, 0.7)
            if passages[x][y][EAST] or (x + 1 < width and passages[x + 1][y][WEST]):
                canvas.begin_path()
                canvas.move_to(x + 0.5, y + 0.5)
                canvas.line_to(x + 1.5, y + 0.5)
                canvas.stroke()
                canvas.close_path()
                # draw_square(x + 1, y + 0.5, 0.7)
    canvas.begin_path()
    canvas.move_to(0.5, 0)
    canvas.line_to(0.5, 0.5)
    canvas.move_to(width - 0.5, width - 0.5)
    canvas.line_to(width, width - 0.5)
    canvas.stroke()
    canvas.close_path()

def expand_location(passages, unexplored, n, here, direction):
    there = np.zeros(2, dtype=int)
    there[X] = here[X] + OFFSETS[direction][X]
    there[Y] = here[Y] + OFFSETS[direction][Y]  
    if contains(there, unexplored, n) != -1:
        add_passage(passages, here, there)
        return there
    else:
        return None

def remove(pair, list, n):
    for i in range(n):
        if ((pair[0] == list[i][0]) and (pair[1] == list[i][1])):
            list[i][0] = list[n-1][0]
            list[i][1] = list[n-1][1]

def expand_maze(passages, done, frontier, unexplored, counts, last_explored_location):
    here = np.zeros(2, dtype=int)
    if last_explored_location is None:
        here = choose_randomly_from(frontier, counts[1])
    else:
        here = last_explored_location
    direction = random.randint(0, 3)
    for i in range(4):
        there = expand_location(passages, unexplored, counts[2], here, direction)
        if there is not None:
            frontier[counts[1]] = there
            counts[1] += 1
            remove(there, unexplored, counts[2])
            counts[2] -= 1
            return there
        direction = (direction + 1) % 4
    done[counts[0]] = here
    counts[0] += 1
    remove(here, frontier, counts[1])
    counts[1] -= 1
    return None

def draw_solution(path, width):
    canvas.line_width = 19.2 / 25.6 / 10
    canvas.stroke_style = 'black'
    canvas.begin_path()
    canvas.move_to(0.5, 0)
    canvas.line_to(0.5, 0.5)
    canvas.move_to(width - 0.5, width - 0.5)
    canvas.line_to(width, width - 0.5)
    canvas.stroke()
    canvas.close_path()
    for i in range(len(path) - 1):
        canvas.begin_path()
        canvas.move_to(path[i][0] + 0.5, path[i][1] + 0.5)
        canvas.line_to(path[i + 1][0] + 0.5, path[i + 1][1] + 0.5)
        canvas.stroke()
        canvas.close_path()

def solve(passages, start, goal):
        if ((start[0] == goal[0]) and (start[1] == goal[1])):
            return [goal]
        for d in range(4):
            if (passages[start[0]][start[1]][d]):
                next = [start[X] + OFFSETS[d][X], start[Y] + OFFSETS[d][Y]]
                restOfPath = solve(passages, next, goal)
                if (restOfPath is not None):
                    return add_to_front(start, restOfPath)
        return None
    
def draw_square(x, y, width):
    canvas.fill_style = 'white'
    canvas.fill_rect(x, y, width)
    
def main():
    width = 20
    canvas.clear()
    canvas.fill_style = 'pink'
    canvas.fill_rect(0, 0, 512, 512)
    # passage_width = 512 / 32
    # wall_width = passage_width / 2
    canvas.scale(25.6, 25.6)
    passages = np.full((width, width, 4), False)
    done = np.full(width * width, None)
    frontier = np.full(width * width, None)
    frontier[0] = np.zeros(2, dtype=int)
    unexplored = np.full(width * width, None)
    counts = [0, 1, width * width - 1]
    i = 0
    for x in range(width):
        for y in range(width):
            # draw_square(x * (passage_width + wall_width) + wall_width, y * (passage_width + wall_width) + wall_width, passage_width)
            if (x != 0 or y != 0):
                unexplored[i] = [x, y]
                i += 1
    last_explored_location = None
    while (counts[2] > 0):
        last_explored_location = expand_maze(passages, done, frontier, unexplored, counts, last_explored_location)
    draw_maze(passages)
    
    solution = solve(passages, [0, 0], [width - 1, width - 1])
    draw_solution(solution, width)
    
main()
canvas

Canvas(height=512, width=512)